# The Cheese


![API Endgoal](./static/simple-api-github.png "API Endgoal")

## Backend Startup

Base folder: `mkdir ember-django && cd ember-django`

Create backend environment with packages: 
- `mkdir events-backend && cd events-backend`
- `pipenv install django djangorestframework djangorestframework-jsonapi`

Activate Backend Shell: `pipenv shell`

Start Django Project: `django-admin startproject main_project .`

Create Events App: `python manage.py startapp events`

Start Server: `python manage.py runserver`

Cool - can't really do anything yet tho 😎

### We end up with a file structure like this:

```
▾ events/
  ▸ migrations/
  - __init__.py
  - admin.py
  - apps.py
  - models.py
  - tests.py
  - views.py
▾ main_project/
  - __init__.py
  - settings.py
  - urls.py
  - wsgi.py
- manage.py
- Pipfile
- Pipfile.lock
```

### Housekeeping: Inject Apps into `main_project`

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    # import rest_framework for viewset and serializer helpers
    'rest_framework',
    # add `events` application to main project
    'events'
]
```

# COMMIT

Backend django project and events app created

## Building Model, Serializer, View and URL for Events

To build an API endpoint (URL to GET, POST, PUT and DELETE) we will do the following:

1. Create our new model (and make migrations)
2. Create a serializer for events
3. Create views for our events
4. Route up our URLs to our views

### Create our Events Model

Add the following code to: `/events-backend/event/models.py`

```python
from django.db import models
from django.utils import timezone

# basic information for our events 
class Event(models.Model):
    # Only required field is title
    title = models.CharField(max_length=256)
    presenter = models.CharField(max_length=256, blank=True)
    # Default time is now if not provided
    time = models.DateTimeField(default=timezone.now)
    location = models.CharField(max_length=256, blank=True)
    description = models.TextField(blank=True)
```

### And migrate the database:

`python manage.py makemigrations`

`python manage.py migrate`

### Make Serializer for Events

Create the file `/events-backend/event/serializers.py` file in events and create serializer:

```python
from django.contrib.auth.models import User, Group
from .models import Event
from rest_framework import serializers


# serializers to convert from Django model objects to JSON
class UserSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = User
        fields = ('url', 'username', 'email', 'groups')

# or specify all fields
class EventSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Event
        fields = '__all__'
```

### Creating a View for our Events

Navigate to the `views.py` file in our events app. Create viewsets:

```python
from django.contrib.auth.models import User
from rest_framework import viewsets
from .models import Event
from .serializers import UserSerializer, EventSerializer


class UserViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows users to be viewed or edited.
    """
    queryset = User.objects.all().order_by('-date_joined')
    serializer_class = UserSerializer

class EventViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows groups to be viewed or edited.
    """
    resource_name = 'events'
    queryset = Event.objects.all()
    serializer_class = EventSerializer
```

### Connecting our URLs

In `events-backend/main_project/urls.py`, register the EventViewSet and the UserViewSet

```python
from django.urls import include, path
from django.contrib import admin
from rest_framework import routers
from events import views

router = routers.DefaultRouter()
router.register(r'users', views.UserViewSet)
router.register(r'events', views.EventViewSet)

urlpatterns = [
    path('api/', include(router.urls)),
    path('api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    path('admin/', admin.site.urls),
]
```

## Start Server and Witness the Magic DRF API

Run: `python manage.py runserver`

And navigate to Events API: http://localhost:8000/api/events/

# COMMIT

Backend API endpoint created for events

# What's a serializer doin anyhow?!


![Serialization from Django Model to JSON](./static/drf-serializer-figure.png "Serialization from Django Model to JSON")

# Recap of Backend


![The parts of our django project](./static/django-architecture.png "The parts of our django project")

### What's in this diagram that we have not yet incorperated into our django project?

# Our Data on JSON-API

JSON is awesome - it is very flexible... 🤔 too flexible for it's own good

JSON-API Specification helps us know what to expect!
> By following shared conventions, you can increase productivity, take advantage of generalized tooling, and focus on what matters: your application.

Also, Ember data expects JSON-API. Convenient! No munging Django Models --> Ember Models



![Side by Side comparison of JSON vs JSON API](static/json-vs-jsonapi.png "Side by Side comparison of JSON vs JSON API")

While JSON-API is more verbose, the consisten structure and the information provided is much more useful for microservice communication.

### Activate JSON-API in our Django App

We have already installed DRF JSON-API at the start. Paste the following code in `main_project/settings.py` to set up JSON-API:

```python
# RestFramework settings for DjangoRestFramework-JSONAPI
REST_FRAMEWORK = {
  'PAGE_SIZE': 100,

  'EXCEPTION_HANDLER':
    'rest_framework_json_api.exceptions.exception_handler',

  'DEFAULT_PAGINATION_CLASS':    'rest_framework_json_api.pagination.JsonApiPageNumberPagination',
  'DEFAULT_PARSER_CLASSES': (
    'rest_framework_json_api.parsers.JSONParser',
    'rest_framework.parsers.FormParser',
    'rest_framework.parsers.MultiPartParser'
  ),
  'DEFAULT_RENDERER_CLASSES': (
    'rest_framework_json_api.renderers.JSONRenderer',
    'rest_framework.renderers.BrowsableAPIRenderer',
   ),
   'DEFAULT_METADATA_CLASS': 'rest_framework_json_api.metadata.JSONAPIMetadata',
   'DEFAULT_FILTER_BACKENDS': (
     'rest_framework.filters.OrderingFilter',
    ),
   'ORDERING_PARAM': 'sort',

   'TEST_REQUEST_RENDERER_CLASSES': (
     'rest_framework_json_api.renderers.JSONRenderer',
    ),

   'TEST_REQUEST_DEFAULT_FORMAT': 'vnd.api+json'
}
```

You can refresh the API in your browser and see the data format has changed: http://localhost:8000/api/events

# The Cheese


![API Endgoal](./static/simple-api-github.png "API Endgoal")

## Frontend Startup

In base folder: `ember-django`

Install Ember: `npm install -g ember-cli`

Create App: `ember new events-frontend`

Enter into Directory: `cd events-frontend`

**Note**: Ember already creates a git repository. You can delete it with `rm -rf .git` if you plan to use a sinple repo in the base folder

## Well Hello, Tomster!
![Tomster Welcome](static/tomster_welcome.png "Ember Welcome Page")

# Load in data - the classic "GET"

To get our data from the API, we will need to:
1. Set up the API URL in `adapter/application.js`
2. Create a model for our event data
3. Fetch our data in our route javascript file
4. Display the data in the browser

## Set up the API URL in `adapter/application.js`

Creat our adapter with Ember CLI: `ember g adapter application`

In the file `app/adapters/application.js` update with:

```javascript
import DS from 'ember-data';
import { computed } from '@ember/object';

export default DS.RESTAdapter.extend({
  host: computed(function(){
    return 'http://localhost:8000';
  }),
  namespace: 'api',
  buildURL: function(type, id, record) {
  //call the default buildURL and then append a slash
  return this._super(type, id, record) + '/';
  }
});
```

Builds URL with: `{ host }` / `{ namespace }` / `{extra bits }` `{append /}` 
     
  --> `http://localhost:8000/` `api/` `events/1` `/`